In [1]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_size = 20000

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json
    


--2019-12-14 16:04:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.26.176, 2404:6800:4007:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.26.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  1.08MB/s    in 8.0s    

2019-12-14 16:04:27 (685 KB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [4]:
with open('/tmp/sarcasm.json','r') as f:
    datastore = json.load(f)
    
sentences = []
labels = []
urls = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [5]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [7]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,maxlen=max_length,truncating=trunc_type,padding=padding_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type,padding=padding_type)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 16)            160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 30
model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

Train on 20000 samples, validate on 6709 samples
Epoch 1/30
20000/20000 [==============================] - 5s 267us/sample - loss: 0.1650 - acc: 0.9379 - val_loss: 0.3783 - val_acc: 0.8526
Epoch 2/30
20000/20000 [==============================] - 7s 329us/sample - loss: 0.1406 - acc: 0.9494 - val_loss: 0.4239 - val_acc: 0.8393
Epoch 3/30
20000/20000 [==============================] - 4s 201us/sample - loss: 0.1215 - acc: 0.9582 - val_loss: 0.4434 - val_acc: 0.8444
Epoch 4/30
20000/20000 [==============================] - 4s 194us/sample - loss: 0.1058 - acc: 0.9642 - val_loss: 0.4874 - val_acc: 0.8384
Epoch 5/30
20000/20000 [==============================] - 4s 183us/sample - loss: 0.0914 - acc: 0.9696 - val_loss: 0.5161 - val_acc: 0.8372
Epoch 6/30
20000/20000 [==============================] - 4s 207us/sample - loss: 0.0810 - acc: 0.9740 - val_loss: 0.5610 - val_acc: 0.8346
Epoch 7/30
20000/20000 [==============================] - 4s 200us/sample - loss: 0.0706 - acc: 0.9778 - val_lo

In [12]:
import matplotlib.pyplot as plt

def plot_graphs(history,string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    
    plt.xlabel('Epochs')
    plt.ylabel(string)
    
    plt.legend([string,'val_'+string])
    plt.show()
    
plot_graphs(history,'acc')
plot_graphs(history,'loss')

NameError: name 'history' is not defined